In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.set_printoptions(linewidth=200, threshold=100000)
torch.cuda.is_available()

True

In [2]:
from src.vlstm_fwbw_v0.interface import vlstm_fw_torch, vlstm_fwbw_cuda
from src.vlstm_fwbw_v0.interface import vlstm_fw_cuda
from vlstm_formulations.vlstm_parallel import vlstm_parallel_fw_torch
# from vlstm_formulations.vlstm_parallel_tiled import vlstm_parallel_tiled
from src.vlstm_fwbw_v0.torch_impl import vlstm_fw_tiled_torch

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt220cu121/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt220cu121/lib


Using /home/max/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu121/vlstm_fwbw_v0/build.ninja...
Building extension module vlstm_fwbw_v0...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /home/max/miniconda3/envs/xlstmpt220cu121/bin/x86_64-conda-linux-gnu-c++ -MMD -MF interface.o.d -DTORCH_EXTENSION_NAME=vlstm_fwbw_v0 -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH -isystem /home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC -isystem /home/max/miniconda3/envs/xlstmpt220cu121/include -isystem /home/max/miniconda3/envs/xlstmpt220cu121/include/python3.11 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /home/max/myrepos/vlstm_cuda/src/vlstm_fwbw_v0/interface.cc -o interface.o 
[2/2] /home/max/miniconda3/envs/xlstmpt220cu121/bin/x86_64-conda-li

Loading extension module vlstm_fwbw_v0...


## CUDA vLSTM forward

In [3]:
S = 64 #32 #16 #8 # seq len
B = 2 # batch size
NH = 1 # num heads
DH = 8 # dim per head
DTYPE = torch.float32
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
# fixed:
qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.

# random: 
# qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1) / 10.
# igs = torch.zeros((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
igs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
fgs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
# fgs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
qs, qs.shape, len(qs.view(-1))

(tensor([[[[0.0000e+00, 1.0000e-01, 2.0000e-01, 3.0000e-01, 4.0000e-01, 5.0000e-01, 6.0000e-01, 7.0000e-01],
           [8.0000e-01, 9.0000e-01, 1.0000e+00, 1.1000e+00, 1.2000e+00, 1.3000e+00, 1.4000e+00, 1.5000e+00],
           [1.6000e+00, 1.7000e+00, 1.8000e+00, 1.9000e+00, 2.0000e+00, 2.1000e+00, 2.2000e+00, 2.3000e+00],
           [2.4000e+00, 2.5000e+00, 2.6000e+00, 2.7000e+00, 2.8000e+00, 2.9000e+00, 3.0000e+00, 3.1000e+00],
           [3.2000e+00, 3.3000e+00, 3.4000e+00, 3.5000e+00, 3.6000e+00, 3.7000e+00, 3.8000e+00, 3.9000e+00],
           [4.0000e+00, 4.1000e+00, 4.2000e+00, 4.3000e+00, 4.4000e+00, 4.5000e+00, 4.6000e+00, 4.7000e+00],
           [4.8000e+00, 4.9000e+00, 5.0000e+00, 5.1000e+00, 5.2000e+00, 5.3000e+00, 5.4000e+00, 5.5000e+00],
           [5.6000e+00, 5.7000e+00, 5.8000e+00, 5.9000e+00, 6.0000e+00, 6.1000e+00, 6.2000e+00, 6.3000e+00],
           [6.4000e+00, 6.5000e+00, 6.6000e+00, 6.7000e+00, 6.8000e+00, 6.9000e+00, 7.0000e+00, 7.1000e+00],
           [7.2000e

In [5]:
hs_pt, n_pt, m_pt, _, matD_pt = vlstm_fw_torch(queries=qs, keys=ks, values=vs, igate_preact=igs, fgate_preact=fgs)

In [6]:
# result_gt, m, l, n = vlstm_fw_tiled_torch(queries=qs, keys=ks, values=vs, igate_preact=igs, fgate_preact=fgs, bq_tile_size=8, bkv_tile_size=8)
# result_gt, result_gt.shape, torch.exp(-m), l, n

In [7]:
# torch.nn.functional.logsigmoid(fgs).squeeze()[17] #+ (10.706538)
# (fgs).squeeze()[17] #+ (10.706538)

In [8]:
# cuda kernel
hs_cuda, n_cuda, m_cuda, matD_cuda = vlstm_fw_cuda(mat_Q=qs, mat_K=ks, mat_V=vs, igate_preact=igs.squeeze(-1), fgate_preact=fgs.squeeze(-1))

before kernel dispatch - float32!
B: 2, NH: 1, S: 64, DH: 8
blocksxy: 2-2, threadsxy: 4-4, shared_mem in bytes: 5664
In FW-Kernel: gdim.x: 2, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In FW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4


In [11]:
RTOL = 1e-10
ATOL = 4e-5
print(f"hs match: {torch.allclose(hs_cuda, hs_pt, rtol=RTOL, atol=ATOL)}")
print(f"n match: {torch.allclose(n_cuda, n_pt, rtol=RTOL, atol=ATOL)}")
print(f"m match: {torch.allclose(m_cuda, m_pt, rtol=RTOL, atol=ATOL)}")
print(f"D match: {torch.allclose((matD_cuda - matD_pt).tril(), torch.zeros_like((matD_cuda)), rtol=RTOL, atol=ATOL)}")
# print(f"delta FBar match: {torch.allclose(dFgs_cu.unsqueeze(-1), dFgs_pt, rtol=RTOL, atol=ATOL)}")

hs match: True
n match: True
m match: True
D match: True


In [10]:
(matD_cuda - matD_pt).tril().abs().max()

tensor(1.4305e-05, device='cuda:0')

In [12]:
n_cuda.shape, n_pt.shape, m_cuda.shape, m_pt.shape

(torch.Size([2, 1, 64, 1]),
 torch.Size([2, 1, 64, 1]),
 torch.Size([2, 1, 64, 1]),
 torch.Size([2, 1, 64, 1]))

In [11]:
matD_pt[:, :, 0:64, :16]

tensor([[[[-9.2466e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf],
          [-1.2379e+00, -4.2534e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf],
          [-1.5512e+00, -7.3861e-01, -2.6438e+00,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf],
          [-1.8644e+00, -1.0519e+00, -2.9571e+00,  1.4518e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf],
          [-2.1777e+00, -1.3651e+00, -3.2704e+00, -1.6808e-01, -1.2087e-01, 

In [12]:
(matD_cuda - matD_pt)[:, :, 0:64, -10:]

tensor([[[[        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
          [        inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf,         inf],
      

In [13]:
hs_cuda - hs_pt

tensor([[[[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11],
          [-2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11, -2.9104e-11],
          [-1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10, -1.1642e-10],
          [ 2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10],
          [ 2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10,  2.3283e-10],
          [-4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [-4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-10, -4.6566e-

In [14]:
n_cuda.unsqueeze(-1) - n_pt

tensor([[[[ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-1.1921e-07],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-1.1921e-07],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-5.9605e-08],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-4.7684e-07],
          [-3.5763e-07],
          [-7.1526e-07],
          [-7.1526e-07],
          [-4.7684e-07],
          [-3.5763e-07],
          [-3.5763e-07],
          [-4.7684e-07],
          [-2.3842e-07],
          [-5.0068e-06],
          [-6.1989e-06],
          [-5.9605e-07],
          [ 2.3842e-07],
          [ 0.0000e+00],
          [-3.5763e-07],
          [-1.1921e-07],
          [ 5.9605e-07],
          [ 4.7684e-07],
          [ 0.0000e+00],
          [-9.5367e-07],
          [-9.5367e-07],
          [-2.8610e-06],
          [ 4.2915e-06],
          [ 2.3842e-06],
          [ 1.0729e-06],


In [15]:
m_cuda.unsqueeze(-1) - m_pt

tensor([[[[ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 5.9605e-08],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 5.9605e-08],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-1.1921e-07],
          [-2.3842e-07],
          [ 0.0000e+00],
          [-1.7881e-07],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-5.9605e-07],
          [-1.3113e-06],
          [-1.9670e-06],
          [-2.1458e-06],
          [-2.3842e-06],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [-1.5497e-06],
          [-1.3113e-06],
          [-5.9605e-08],
          [ 0.0000e+00],
          [-7.1526e-07],
          [-3.5763e-07],
          [ 8.9407e-07],
          [ 1.1921e-06],
          [ 1.4305e-06],
          [ 1.6689e-06],
          [ 0.0000e+00],
          [ 0.0000e+00],
          [ 0.0000e+00],


In [16]:
matD_cuda.shape

torch.Size([2, 1, 64, 64])

In [17]:
matD_cuda[:, :, :10, :10]

tensor([[[[-0.9247,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,  0.0000,  0.0000],
          [-1.2379, -0.4253,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,  0.0000,  0.0000],
          [-1.5512, -0.7386, -2.6438,    -inf,    -inf,    -inf,    -inf,    -inf,  0.0000,  0.0000],
          [-1.8644, -1.0519, -2.9571,  0.1452,    -inf,    -inf,    -inf,    -inf,  0.0000,  0.0000],
          [-2.1777, -1.3651, -3.2704, -0.1681, -0.1209,    -inf,    -inf,    -inf,  0.0000,  0.0000],
          [-2.4910, -1.6784, -3.5836, -0.4813, -0.4341, -0.5797,    -inf,    -inf,  0.0000,  0.0000],
          [-2.8042, -1.9917, -3.8969, -0.7946, -0.7474, -0.8930, -0.6229,    -inf,  0.0000,  0.0000],
          [-3.1175, -2.3049, -4.2102, -1.1079, -1.0607, -1.2062, -0.9361, -0.3284,  0.0000,  0.0000],
          [-3.4308, -2.6182, -4.5234, -1.4211, -1.3739, -1.5195, -1.2494, -0.6416, -1.0745,    -inf],
          [-3.7440, -2.9314, -4.8367, -1.7344, -1.6872, -1.8328, -1.5626, -0.9549,

In [18]:
matD_pt

tensor([[[[-9.2466e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,
                  -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf,        -inf],
          [-1.2379e+00, -4.2534e-01,        -inf,        -inf,        -inf,        -inf,        -inf,        -in

In [19]:
(matD_cuda - matD_pt)[:, :, :32, :10]

tensor([[[[ 0.0000e+00,         nan,         nan,         nan,         nan,         nan,         nan,         nan,         inf,         inf],
          [ 0.0000e+00,  0.0000e+00,         nan,         nan,         nan,         nan,         nan,         nan,         inf,         inf],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,         nan,         nan,         nan,         nan,         nan,         inf,         inf],
          [ 1.1921e-07,  1.1921e-07,  0.0000e+00,  0.0000e+00,         nan,         nan,         nan,         nan,         inf,         inf],
          [ 2.3842e-07,  2.3842e-07,  2.3842e-07,  5.9605e-08,  0.0000e+00,         nan,         nan,         nan,         inf,         inf],
          [ 2.3842e-07,  2.3842e-07,  2.3842e-07,  1.1921e-07,  5.9605e-08,  0.0000e+00,         nan,         nan,         inf,         inf],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  5.9605e-08,  0.0000e+00, -5.9605e-08,  0.0000e+00,         nan,         inf,         inf],
      

In [20]:
matD_pt[:, :, :10, :10]

tensor([[[[-0.9247,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [-1.2379, -0.4253,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [-1.5512, -0.7386, -2.6438,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [-1.8644, -1.0519, -2.9571,  0.1452,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf],
          [-2.1777, -1.3651, -3.2704, -0.1681, -0.1209,    -inf,    -inf,    -inf,    -inf,    -inf],
          [-2.4910, -1.6784, -3.5836, -0.4813, -0.4341, -0.5797,    -inf,    -inf,    -inf,    -inf],
          [-2.8042, -1.9917, -3.8969, -0.7946, -0.7474, -0.8930, -0.6229,    -inf,    -inf,    -inf],
          [-3.1175, -2.3049, -4.2102, -1.1079, -1.0607, -1.2062, -0.9361, -0.3284,    -inf,    -inf],
          [-3.4308, -2.6182, -4.5234, -1.4211, -1.3739, -1.5195, -1.2494, -0.6416, -1.0745,    -inf],
          [-3.7440, -2.9314, -4.8367, -1.7344, -1.6872, -1.8328, -1.5626, -0.9549,